## Import libraries

In [20]:
import pandas as pd
import requests
import tabula

## Import Consumer Expenditure Surveys, U.S. Bureau of Labor Statistics, 2019-2020 (Northeast US)

In [42]:
#using tabula to parse pdf
dfs = tabula.read_pdf("https://www.bls.gov/cex/tables/cross-tab/mean/cu-region-by-income-northeast-2020.pdf", pages = "all")

## Find Income + Total Expenditures of Income Groups
- Low-income group is population of households that earns between $15,000 - $30,000.
- High-income group is population of households that earns between $100,000 - $149,999.

In [43]:
#select columns of income groups under analysis
dfSES = dfs[0][['Unnamed: 0','$15,000','$100,000']]
dfSES.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

dfIncomeExpense = dfSES.loc[[9,49]]
dfIncomeExpense['lowIncomeExpense'] = dfIncomeExpense['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfIncomeExpense['highIncomeExpense'] = dfIncomeExpense['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)

dfIncomeExpense = dfIncomeExpense.T
dfIncomeExpense.columns = ['income','expenses']
dfIncomeExpense = dfIncomeExpense[1:]
dfIncomeExpense.index = ["lowIncome","highIncome"]
dfIncomeExpense


,income,expenses
lowIncome,22498.0,34970.0
highIncome,121558.0,85111.0


## Find Expenditure Categories From the Consumer Expenditure Surveys
- Pick out major expenditure categories that match the BLS' Consumer Price Index estimates and calculate how much each income group spent on the various categories. This is required in order to estimate their inflationary cost in dollar amounts.

In [44]:
dfExpense = pd.concat(dfs).reset_index(drop=True)[['Unnamed: 0','$15,000','$100,000']]
dfExpense.columns = ['category', "lowIncomeExpense","highIncomeExpense"]

#These rows correspond to major category groups that match up to BLS' inflation categories.
dfExpenseToMatch = dfExpense.loc[[51,52,83,85,88,96,105,108,112,120,131,145,151,169,165,167,171,173,175,177]]
dfExpenseToMatch['lowIncomeExpense'] = dfExpenseToMatch['lowIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['highIncomeExpense'] = dfExpenseToMatch['highIncomeExpense'].str.replace(",","").str.replace("$","",regex=False).astype(float)
dfExpenseToMatch['category'] = dfExpenseToMatch['category'].str.replace(".","",regex=False).str.rstrip()
dfExpenseToMatch = dfExpenseToMatch.reset_index(drop=True)

#rename and group categories
dfExpenseToMatch = dfExpenseToMatch.replace("Household operations","Household furnishings and operations",regex=True) \
                                   .replace("Housekeeping supplies","Household furnishings and operations",regex=True) \
                                   .replace("Household furnishings and equipment","Household furnishings and operations",regex=True) \
                                   .replace("Reading","Other goods and services",regex=True) \
                                   .replace("Tobacco products and smoking supplies","Other goods and services",regex=True) \
                                   .replace("Miscellaneous","Other goods and services",regex=True) \
                                   .replace("Cash contributions","Other goods and services",regex=True) \
                                   .replace("Personal insurance and pensions","Other goods and services",regex=True)

#hack for string issue with one expenditure category?
dfExpenseToMatch = dfExpenseToMatch.replace("Personal care products and services",dfExpenseToMatch['category'][13],regex=True)
dfExpenseToMatch = dfExpenseToMatch.groupby("category").sum().reset_index()


## Import Consumer Price Index, U.S. Bureau of Labor Statistics, Northeast Region – May 2022
- Get inflation rate figures from monethly BLS CPI estimates for various categories and match them to the categories in the BLS Consumer Expenditure Survey.

In [45]:
tablesInflation = pd.read_html('https://www.bls.gov/regions/mid-atlantic/cpi-summary/ro3xg01a.htm') # or alternatively https://www.bls.gov/regions/mid-atlantic/news-release/consumerpriceindex_northeast.htm
dfInflation = tablesInflation[0]
dfInflation = dfInflation['Item']['Item'].to_frame().join(dfInflation['Northeast region']['Percent change fromMay2021'])

In [46]:
#pick out categories to match CES
dfInflationToMatch = dfInflation.loc[[3,4,11,12,14,18,23,24,25,33,34,35,37]].reset_index(drop=True)
dfInflationToMatch.columns = ['category','inflationRate']

#rename rows
dfInflationToMatch['category'].loc[5] = "Utilities, fuels, and public services"
dfInflationToMatch['category'].loc[7] = "Apparel and services"
dfInflationToMatch['category'].loc[9] = "Healthcare"
dfInflationToMatch['category'].loc[10] = "Entertainment"
dfInflationToMatch['category'].loc[11] = "Education"
dfInflationToMatch['inflationRate'] = dfInflationToMatch['inflationRate'].astype(float)
dfInflationToMatch

,category,inflationRate
0,Food,9.0
1,Food at home,10.4
2,Food away from home,6.3
3,Alcoholic beverages,4.9
4,Shelter,3.2
5,"Utilities, fuels, and public services",26.3
6,Household furnishings and operations,9.0
7,Apparel and services,3.7
8,Transportation,16.9
9,Healthcare,3.4


## Merge Inflation + Expense Data, Estimate Total Inflation Cost
- Merge inflation and expense datasets, calculate additional expenditure due to inflation for each category.

In [47]:
dfInflationExpense = pd.merge(dfInflationToMatch,dfExpenseToMatch,on="category")

In [48]:
dfInflationExpense['lowIncomeInflation'] = dfInflationExpense['lowIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense['highIncomeInflation'] = dfInflationExpense['highIncomeExpense'] *dfInflationExpense['inflationRate'] / 100
dfInflationExpense

,category,inflationRate,lowIncomeExpense,highIncomeExpense,lowIncomeInflation,highIncomeInflation
0,Food,9.0,4667.0,10904.0,420.030,981.360
1,Food at home,10.4,3323.0,6759.0,345.592,702.936
2,Food away from home,6.3,1345.0,4145.0,84.735,261.135
3,Alcoholic beverages,4.9,212.0,938.0,10.388,45.962
4,Shelter,3.2,10963.0,16537.0,350.816,529.184
5,"Utilities, fuels, and public services",26.3,2906.0,5264.0,764.278,1384.432
6,Household furnishings and operations,9.0,2326.0,5459.0,209.340,491.310
7,Apparel and services,3.7,996.0,3077.0,36.852,113.849
8,Transportation,16.9,4058.0,13705.0,685.802,2316.145
9,Healthcare,3.4,3653.0,6318.0,124.202,214.812


In [49]:
#exclude first row (Food) since next two rows are components of that.
dfIncomeExpense['inflation'] = [dfInflationExpense['lowIncomeInflation'][1:].sum().round(),dfInflationExpense['highIncomeInflation'][1:].sum().round()]
dfIncomeExpenseInflation = dfIncomeExpense

In [50]:
dfIncomeExpenseInflation

,income,expenses,inflation
lowIncome,22498.0,34970.0,2882.0
highIncome,121558.0,85111.0,7273.0


## Calculate Inflation as Portion of Income
- Take the additional inflation expenditures of each category as a proportion of an income group's average income to find the percentage of an income's groups income that has been taken up by inflationary pressure for certain categories.

In [32]:
#pick out relevant rows
dfInflationIncome = dfInflationExpense.loc[[0,8,4,5,9]]

#take proportion and round
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflation'] / dfIncomeExpense['income']['lowIncome'] * 100
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflation'] / dfIncomeExpense['income']['highIncome'] * 100
dfInflationIncome['lowIncomeInflationProp'] = dfInflationIncome['lowIncomeInflationProp'].round(1)
dfInflationIncome['highIncomeInflationProp'] = dfInflationIncome['highIncomeInflationProp'].round(1)

dfInflationIncome = dfInflationIncome[['category','inflationRate','lowIncomeInflationProp','highIncomeInflationProp']]
dfInflationIncome



,category,inflationRate,lowIncomeInflationProp,highIncomeInflationProp
0,Food,9.0,1.9,0.8
8,Transportation,16.9,3.0,1.9
4,Shelter,3.2,1.6,0.4
5,"Utilities, fuels, and public services",26.3,3.4,1.1
9,Healthcare,3.4,0.6,0.2


## Get Other Inflation Rates We Care About
- These are other inflation rates that are refernced in the story.

In [40]:
dfOtherRates = dfInflation.loc[[0,3,14,31,21,22]]
dfOtherRates = dfOtherRates.reset_index(drop=True)
dfOtherRates.columns = ['category','inflationRate']

In [41]:
dfOtherRates

,category,inflationRate
0,All items,7.5
1,Food,9.0
2,Shelter,3.2
3,Gasoline,51.9
4,Electricity,15.3
5,Utility (piped) gas service,31.1
